In [1]:
import pathlib
import sqlite3

import numpy as np
import pandas as pd
from cytotable import convert

In [2]:
# set paths
dest_datatype = "parquet"

# common configurations to use based on typical CellProfiler SQLite outputs
preset = "cellprofiler_sqlite_pycytominer"

input_dir = pathlib.Path("../../data/4.sqlite_output")

# directory where parquet files are saved to
output_dir = pathlib.Path("../../data/5.converted_data")
output_dir.mkdir(exist_ok=True)

source_path = input_dir / "OP_features.sqlite"
dest_path = output_dir / f"OP_features.{dest_datatype}"

In [3]:
# Read in the sqlite file via sqlite3
conn = sqlite3.connect(source_path)

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables = tables["name"].values
tables

array(['Per_Object', 'Per_Image', 'Experiment', 'sqlite_sequence',
       'Experiment_Properties', 'Per_Experiment', 'Per_RelationshipTypes',
       'Per_Relationships'], dtype=object)

In [4]:
images_df = pd.read_sql_query(f"SELECT * FROM Per_Image", conn)
# select columns to keep in the final dataframe
columns_to_keep = ["ImageNumber", "Image_FileName_OP"]
images_df = images_df[columns_to_keep]

In [5]:
query = "SELECT * FROM Per_Object"
df = pd.read_sql_query(query, conn)


# combine the two dataframes on ImageNumber
df = df.merge(images_df, on="ImageNumber")
conn.close()

In [6]:
# max columns
pd.set_option("display.max_columns", None)
print(df.shape)
df.head()

(9692, 283)


,ImageNumber,ObjectNumber,ConvertImageToObjects_Number_Object_Number,ConvertImageToObjects_AreaShape_Area,ConvertImageToObjects_AreaShape_BoundingBoxArea,ConvertImageToObjects_AreaShape_BoundingBoxMaximum_X,ConvertImageToObjects_AreaShape_BoundingBoxMaximum_Y,ConvertImageToObjects_AreaShape_BoundingBoxMinimum_X,ConvertImageToObjects_AreaShape_BoundingBoxMinimum_Y,ConvertImageToObjects_AreaShape_Center_X,ConvertImageToObjects_AreaShape_Center_Y,ConvertImageToObjects_AreaShape_CentralMoment_0_0,ConvertImageToObjects_AreaShape_CentralMoment_0_1,ConvertImageToObjects_AreaShape_CentralMoment_0_2,ConvertImageToObjects_AreaShape_CentralMoment_0_3,ConvertImageToObjects_AreaShape_CentralMoment_1_0,ConvertImageToObjects_AreaShape_CentralMoment_1_1,ConvertImageToObjects_AreaShape_CentralMoment_1_2,ConvertImageToObjects_AreaShape_CentralMoment_1_3,ConvertImageToObjects_AreaShape_CentralMoment_2_0,ConvertImageToObjects_AreaShape_CentralMoment_2_1,ConvertImageToObjects_AreaShape_CentralMoment_2_2,ConvertImageToObjects_AreaShape_CentralMoment_2_3,ConvertImageToObjects_AreaShape_Compactness,ConvertImageToObjects_AreaShape_ConvexArea,ConvertImageToObjects_AreaShape_Eccentricity,ConvertImageToObjects_AreaShape_EquivalentDiameter,ConvertImageToObjects_AreaShape_EulerNumber,ConvertImageToObjects_AreaShape_Extent,ConvertImageToObjects_AreaShape_FormFactor,ConvertImageToObjects_AreaShape_HuMoment_0,ConvertImageToObjects_AreaShape_HuMoment_1,ConvertImageToObjects_AreaShape_HuMoment_2,ConvertImageToObjects_AreaShape_HuMoment_3,ConvertImageToObjects_AreaShape_HuMoment_4,ConvertImageToObjects_AreaShape_HuMoment_5,ConvertImageToObjects_AreaShape_HuMoment_6,ConvertImageToObjects_AreaShape_InertiaTensorEigenvalues_0,ConvertImageToObjects_AreaShape_InertiaTensorEigenvalues_1,ConvertImageToObjects_AreaShape_InertiaTensor_0_0,ConvertImageToObjects_AreaShape_InertiaTensor_0_1,ConvertImageToObjects_AreaShape_InertiaTensor_1_0,ConvertImageToObjects_AreaShape_InertiaTensor_1_1,ConvertImageToObjects_AreaShape_MajorAxisLength,ConvertImageToObjects_AreaShape_MaxFeretDiameter,ConvertImageToObjects_AreaShape_MaximumRadius,ConvertImageToObjects_AreaShape_MeanRadius,ConvertImageToObjects_AreaShape_MedianRadius,ConvertImageToObjects_AreaShape_MinFeretDiameter,ConvertImageToObjects_AreaShape_MinorAxisLength,ConvertImageToObjects_AreaShape_NormalizedMoment_0_0,ConvertImageToObjects_AreaShape_NormalizedMoment_0_1,ConvertImageToObjects_AreaShape_NormalizedMoment_0_2,ConvertImageToObjects_AreaShape_NormalizedMoment_0_3,ConvertImageToObjects_AreaShape_NormalizedMoment_1_0,ConvertImageToObjects_AreaShape_NormalizedMoment_1_1,ConvertImageToObjects_AreaShape_NormalizedMoment_1_2,ConvertImageToObjects_AreaShape_NormalizedMoment_1_3,ConvertImageToObjects_AreaShape_NormalizedMoment_2_0,ConvertImageToObjects_AreaShape_NormalizedMoment_2_1,ConvertImageToObjects_AreaShape_NormalizedMoment_2_2,ConvertImageToObjects_AreaShape_NormalizedMoment_2_3,ConvertImageToObjects_AreaShape_NormalizedMoment_3_0,ConvertImageToObjects_AreaShape_NormalizedMoment_3_1,ConvertImageToObjects_AreaShape_NormalizedMoment_3_2,ConvertImageToObjects_AreaShape_NormalizedMoment_3_3,ConvertImageToObjects_AreaShape_Orientation,ConvertImageToObjects_AreaShape_Perimeter,ConvertImageToObjects_AreaShape_Solidity,ConvertImageToObjects_AreaShape_SpatialMoment_0_0,ConvertImageToObjects_AreaShape_SpatialMoment_0_1,ConvertImageToObjects_AreaShape_SpatialMoment_0_2,ConvertImageToObjects_AreaShape_SpatialMoment_0_3,ConvertImageToObjects_AreaShape_SpatialMoment_1_0,ConvertImageToObjects_AreaShape_SpatialMoment_1_1,ConvertImageToObjects_AreaShape_SpatialMoment_1_2,ConvertImageToObjects_AreaShape_SpatialMoment_1_3,ConvertImageToObjects_AreaShape_SpatialMoment_2_0,ConvertImageToObjects_AreaShape_SpatialMoment_2_1,ConvertImageToObjects_AreaShape_SpatialMoment_2_2,ConvertImageToObjects_AreaShape_SpatialMoment_2_3,ConvertImageToObjects_AreaShape_Zernike_0_0,ConvertImageToObjects_AreaShape_Zernike_1_1,ConvertImageToObjects_AreaShape_Z

In [7]:
# get the column names that have BoundingBox in them
bounding_box_cols = [x for x in df.columns if "BoundingBox" in x]
# get the column names that have Location in them
location_cols = [x for x in df.columns if "Location" in x]
# get the column names that have Center in them
center_cols = [x for x in df.columns if "Center" in x]
# manual list
manual_cols = [
    "ImageNumber",
    "Image_FileName_OP",
    "ObjectNumber",
    "ConvertImageToObjects_Number_Object_Number",
]

final_metadata_cols = manual_cols + bounding_box_cols + location_cols + center_cols

In [8]:
# select all of the metadata columns
metadata_df = df[final_metadata_cols]
metadata_df.head()
# add "Metadata to the front of the column names"
metadata_df.columns = ["Metadata_" + x for x in metadata_df.columns]
metadata_df.head()

,Metadata_ImageNumber,Metadata_Image_FileName_OP,Metadata_ObjectNumber,Metadata_ConvertImageToObjects_Number_Object_Number,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxArea,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_X,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_Y,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_X,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_Y,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_X_OP,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Y_OP,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Z_OP,Metadata_ConvertImageToObjects_Location_Center_X,Metadata_ConvertImageToObjects_Location_Center_Y,Metadata_ConvertImageToObjects_Location_Center_Z,Metadata_ConvertImageToObjects_Location_MaxIntensity_X_OP,Metadata_ConvertImageToObjects_Location_MaxIntensity_Y_OP,Metadata_ConvertImageToObjects_Location_MaxIntensity_Z_OP,Metadata_ConvertImageToObjects_AreaShape_Center_X,Metadata_ConvertImageToObjects_AreaShape_Center_Y,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_X_OP,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Y_OP,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Z_OP,Metadata_ConvertImageToObjects_Location_Center_X,Metadata_ConvertImageToObjects_Location_Center_Y,Metadata_ConvertImageToObjects_Location_Center_Z
0,1,MAX_high_10_L.tiff,1,1,19470.0,213.0,175.0,48.0,57.0,117.563212,124.026375,0.0,118.317985,122.647959,0.0,157.0,133.0,0.0,118.317985,122.647959,117.563212,124.026375,0.0,118.317985,122.647959,0.0
1,1,MAX_high_10_L.tiff,2,2,1.0,67.0,82.0,66.0,81.0,66.000000,81.000000,0.0,66.000000,81.000000,0.0,66.0,81.0,0.0,66.000000,81.000000,66.000000,81.000000,0.0,66.000000,81.000000,0.0
2,1,MAX_high_10_L.tiff,3,3,1.0,81.0,107.0,80.0,106.0,80.000000,106.000000,0.0,80.000000,106.000000,0.0,80.0,106.0,0.0,80.000000,106.000000,80.000000,106.000000,0.0,80.000000,106.000000,0.0
3,1,MAX_high_10_L.tiff,4,4,1.0,179.0,114.0,178.0,113.0,178.000000,113.000000,0.0,178.000000,113.000000,0.0,178.0,113.0,0.0,178.000000,113.000000,178.000000,113.000000,0.0,178.000000,113.000000,0.0
4,1,MAX_high_10_L.tiff,5,5,1.0,181.0,120.0,180.0,119.0,180.000000,119.000000,0.0,180.000000,119.000000,0.0,180.0,119.0,0.0,180.000000,119.000000,180.000000,119.000000,0.0,180.000000,119.000000,0.0


In [9]:
# get the ConvertImageToObjects columns and remove the ConvertImageToObjects_ prefix
convert_cols = [x for x in df.columns if "ConvertImageToObjects" in x]
convert_df = df[convert_cols]
convert_df.columns = [
    x.replace("ConvertImageToObjects_", "") for x in convert_df.columns
]
convert_df.head()

,Number_Object_Number,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,AreaShape_CentralMoment_1_3,AreaShape_CentralMoment_2_0,AreaShape_CentralMoment_2_1,AreaShape_CentralMoment_2_2,AreaShape_CentralMoment_2_3,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_EulerNumber,AreaShape_Extent,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,AreaShape_HuMoment_4,AreaShape_HuMoment_5,AreaShape_HuMoment_6,AreaShape_InertiaTensorEigenvalues_0,AreaShape_InertiaTensorEigenvalues_1,AreaShape_InertiaTensor_0_0,AreaShape_InertiaTensor_0_1,AreaShape_InertiaTensor_1_0,AreaShape_InertiaTensor_1_1,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,AreaShape_MaximumRadius,AreaShape_MeanRadius,AreaShape_MedianRadius,AreaShape_MinFeretDiameter,AreaShape_MinorAxisLength,AreaShape_NormalizedMoment_0_0,AreaShape_NormalizedMoment_0_1,AreaShape_NormalizedMoment_0_2,AreaShape_NormalizedMoment_0_3,AreaShape_NormalizedMoment_1_0,AreaShape_NormalizedMoment_1_1,AreaShape_NormalizedMoment_1_2,AreaShape_NormalizedMoment_1_3,AreaShape_NormalizedMoment_2_0,AreaShape_NormalizedMoment_2_1,AreaShape_NormalizedMoment_2_2,AreaShape_NormalizedMoment_2_3,AreaShape_NormalizedMoment_3_0,AreaShape_NormalizedMoment_3_1,AreaShape_NormalizedMoment_3_2,AreaShape_NormalizedMoment_3_3,AreaShape_Orientation,AreaShape_Perimeter,AreaShape_Solidity,AreaShape_SpatialMoment_0_0,AreaShape_SpatialMoment_0_1,AreaShape_SpatialMoment_0_2,AreaShape_SpatialMoment_0_3,AreaShape_SpatialMoment_1_0,AreaShape_SpatialMoment_1_1,AreaShape_SpatialMoment_1_2,AreaShape_SpatialMoment_1_3,AreaShape_SpatialMoment_2_0,AreaShape_SpatialMoment_2_1,AreaShape_SpatialMoment_2_2,AreaShape_SpatialMoment_2_3,AreaShape_Zernike_0_0,AreaShape_Zernike_1_1,AreaShape_Zernike_2_0,AreaShape_Zernike_2_2,AreaShape_Zernike_3_1,AreaShape_Zernike_3_3,AreaShape_Zernike_4_0,AreaShape_Zernike_4_2,AreaShape_Zernike_4_4,AreaShape_Zernike_5_1,AreaShape_Zernike_5_3,AreaShape_Zernike_5_5,AreaShape_Zernike_6_0,AreaShape_Zernike_6_2,AreaShape_Zernike_6_4,AreaShape_Zernike_6_6,AreaShape_Zernike_7_1,AreaShape_Zernike_7_3,AreaShape_Zernike_7_5,AreaShape_Zernike_7_7,AreaShape_Zernike_8_0,AreaShape_Zernike_8_2,AreaShape_Zernike_8_4,AreaShape_Zernike_8_6,AreaShape_Zernike_8_8,AreaShape_Zernike_9_1,AreaShape_Zernike_9_3,AreaShape_Zernike_9_5,AreaShape_Zernike_9_7,AreaShape_Zernike_9_9,Granularity_10_OP,Granularity_11_OP,Granularity_12_OP,Granularity_13_OP,Granularity_14_OP,Granularity_15_OP,Granularity_16_OP,Granularity_1_OP,Granularity_2_OP,Granularity_3_OP,Granularity_4_OP,Granularity_5_OP,Granularity_6_OP,Granularity_7_OP,Granularity_8_OP,Granularity_9_OP,Intensity_IntegratedIntensityEdge_OP,Intensity_IntegratedIntensity_OP,Intensity_LowerQuartileIntensity_OP,Intensity_MADIntensity_OP,Intensity_MassDisplacement_OP,Intensity_MaxIntensityEdge_OP,Intensity_MaxIntensity_OP,Intensity_MeanIntensityEdge_OP,Intensity_MeanIntensity_OP,Intensity_MedianIntensity_OP,Intensity_MinIntensityEdge_OP,Intensity_MinIntensity_OP,Intensity_StdIntensityEdge_OP,Intensity_StdIntensity_OP,Intensity_UpperQuartileIntensity_OP,Location_CenterMassIntensity_X_OP,Location_CenterMassIntensity_Y_OP,Location_CenterMassIntensity_Z_OP,Location_Center_X,Location_Center_Y,Location_Center_Z,Location_MaxIntensity_X_OP,Location_MaxIntensity_Y_OP,Location_MaxIntensity_Z_OP,Neighbors_AngleBetweenNeighbors_Adjacent,Neighbors_FirstClosestDistance_Adjacent,Neighbors_FirstClosestObjectNumber_Adjacnt,Neighbors_NumberOfNeighbors_Adjacent,Neighbors_PercentTouching_Adjacent,Neighbors_SecondClosestDistance_Adjacent,Neighbors_SecondCl

In [14]:
# add the metadata and convert dataframes together
df = pd.concat([metadata_df, convert_df], axis=1)
duplicates = df.columns[df.columns.duplicated()].tolist()
print("Duplicate columns are:", duplicates)
# drop the duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

Duplicate columns are: ['Metadata_ConvertImageToObjects_Location_CenterMassIntensity_X_OP', 'Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Y_OP', 'Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Z_OP', 'Metadata_ConvertImageToObjects_Location_Center_X', 'Metadata_ConvertImageToObjects_Location_Center_Y', 'Metadata_ConvertImageToObjects_Location_Center_Z']


In [15]:
df

,Metadata_ImageNumber,Metadata_Image_FileName_OP,Metadata_ObjectNumber,Metadata_ConvertImageToObjects_Number_Object_Number,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxArea,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_X,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_Y,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_X,Metadata_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_Y,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_X_OP,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Y_OP,Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Z_OP,Metadata_ConvertImageToObjects_Location_Center_X,Metadata_ConvertImageToObjects_Location_Center_Y,Metadata_ConvertImageToObjects_Location_Center_Z,Metadata_ConvertImageToObjects_Location_MaxIntensity_X_OP,Metadata_ConvertImageToObjects_Location_MaxIntensity_Y_OP,Metadata_ConvertImageToObjects_Location_MaxIntensity_Z_OP,Metadata_ConvertImageToObjects_AreaShape_Center_X,Metadata_ConvertImageToObjects_AreaShape_Center_Y,Number_Object_Number,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,AreaShape_CentralMoment_1_3,AreaShape_CentralMoment_2_0,AreaShape_CentralMoment_2_1,AreaShape_CentralMoment_2_2,AreaShape_CentralMoment_2_3,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_EulerNumber,AreaShape_Extent,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,AreaShape_HuMoment_4,AreaShape_HuMoment_5,AreaShape_HuMoment_6,AreaShape_InertiaTensorEigenvalues_0,AreaShape_InertiaTensorEigenvalues_1,AreaShape_InertiaTensor_0_0,AreaShape_InertiaTensor_0_1,AreaShape_InertiaTensor_1_0,AreaShape_InertiaTensor_1_1,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,AreaShape_MaximumRadius,AreaShape_MeanRadius,AreaShape_MedianRadius,AreaShape_MinFeretDiameter,AreaShape_MinorAxisLength,AreaShape_NormalizedMoment_0_0,AreaShape_NormalizedMoment_0_1,AreaShape_NormalizedMoment_0_2,AreaShape_NormalizedMoment_0_3,AreaShape_NormalizedMoment_1_0,AreaShape_NormalizedMoment_1_1,AreaShape_NormalizedMoment_1_2,AreaShape_NormalizedMoment_1_3,AreaShape_NormalizedMoment_2_0,AreaShape_NormalizedMoment_2_1,AreaShape_NormalizedMoment_2_2,AreaShape_NormalizedMoment_2_3,AreaShape_NormalizedMoment_3_0,AreaShape_NormalizedMoment_3_1,AreaShape_NormalizedMoment_3_2,AreaShape_NormalizedMoment_3_3,AreaShape_Orientation,AreaShape_Perimeter,AreaShape_Solidity,AreaShape_SpatialMoment_0_0,AreaShape_SpatialMoment_0_1,AreaShape_SpatialMoment_0_2,AreaShape_SpatialMoment_0_3,AreaShape_SpatialMoment_1_0,AreaShape_SpatialMoment_1_1,AreaShape_SpatialMoment_1_2,AreaShape_SpatialMoment_1_3,AreaShape_SpatialMoment_2_0,AreaShape_SpatialMoment_2_1,AreaShape_SpatialMoment_2_2,AreaShape_SpatialMoment_2_3,AreaShape_Zernike_0_0,AreaShape_Zernike_1_1,AreaShape_Zernike_2_0,AreaShape_Zernike_2_2,AreaShape_Zernike_3_1,AreaShape_Zernike_3_3,AreaShape_Zernike_4_0,AreaShape_Zernike_4_2,AreaShape_Zernike_4_4,AreaShape_Zernike_5_1,AreaShape_Zernike_5_3,AreaShape_Zernike_5_5,AreaShape_Zernike_6_0,AreaShape_Zernike_6_2,AreaShape_Zernike_6_4,AreaShape_Zernike_6_6,AreaShape_Zernike_7_1,AreaShape_Zernike_7_3,AreaShape_Zernike_7_5,AreaShape_Zernike_7_7,AreaShape_Zernike_8_0,AreaShape_Zernike_8_2,AreaShape_Zernike_8_4,AreaShape_Zernike_8_6,AreaShape_Zernike_8_8,AreaShape_Zernike_9_1,AreaShape_Zernike_9_3,AreaShape_Zernike_9_5,AreaShape_Zernike_9_7,AreaShape_Zernike_9_9,Granularity_10_OP,Granularity_11_OP,Granularity_12_OP,Granularity_13_OP,Granularity_14_OP,Granularity_15_OP,Granularity_16_OP,Granularity_1_OP,Granularity_2_OP,Granularity_3_OP,Gran

In [16]:
# write the df to a parquet file
df.to_parquet(dest_path, index=False)

Duplicate columns are: ['Metadata_ConvertImageToObjects_Location_CenterMassIntensity_X_OP', 'Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Y_OP', 'Metadata_ConvertImageToObjects_Location_CenterMassIntensity_Z_OP', 'Metadata_ConvertImageToObjects_Location_Center_X', 'Metadata_ConvertImageToObjects_Location_Center_Y', 'Metadata_ConvertImageToObjects_Location_Center_Z']
